In [ ]:
import sqlite3

import dotenv
import pandas as pd

from open_deep_researcher.retriever.patent import (
    SQLiteFTSPatentetriever,
    generate_patent_search_keywords,
    initialize_patent_database,
    patent_search,
    search_google_patents,
)

dotenv.load_dotenv()

In [ ]:
# test generate_patent_search_keywords
keywords = await generate_patent_search_keywords(topic="光格子時計", llm_provider="openai", llm_model="gpt-4o")
print(keywords)

In [ ]:
# test search_google_patents
keywords_list = keywords.keywords_list + keywords.translated_keywords_list
results = await search_google_patents(keywords_list=keywords_list, limit=100)
print(results[0])

In [ ]:
# test patent db initialization
topic = "光格子時計"
db_path = "tmp/patent_database.sqlite"

success = await initialize_patent_database(
    topic=topic,
    db_path=db_path,
    llm_provider="openai",
    llm_model="gpt-4o",
)
if success:
    print("Database initialized successfully.")

    conn = sqlite3.connect(db_path)
    query = """
    SELECT
        patent_id,
        title,
        abstract,
        publication_date,
        url
    FROM
        patents
    ORDER BY
        publication_date DESC
    """
    df = pd.read_sql_query(query, conn)
    retriever = SQLiteFTSPatentetriever(db_path=db_path)
    retriever.search("atomic clock", limit=5)

    conn.close()
else:
    print("Database initialization failed.")

In [ ]:
# test patent_search
result = await patent_search(
    search_queries=["atomic clock"],
    limit=5,
    db_path=db_path,
)
print(result)

In [6]:
# cleanup
import shutil

shutil.rmtree("tmp")